In [1]:
import pandas as pd
from string import punctuation

In [2]:
df = pd.read_csv('result.tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [3]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)

In [4]:
df = df.dropna()
df = df.drop_duplicates()

In [5]:
def normalize(text):
    
    normalized_text = [(word.strip(punctuation)) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [6]:
leagues_dict = {}
for league in df['league_name_in_russian']:
    league = league.lower()
    leagues_dict[league] = 'лига'

print(len(leagues_dict))

283


In [7]:
newdfleague = pd.DataFrame(list(leagues_dict.items()), columns=['Word', 'Value'])

In [8]:
newdfleague['Value'] = 1

In [9]:
newdfleague.head()

,Word,Value
0,международные товарищеские матчи,1
1,кубок америки,1
2,лига европы уефа,1
3,международный кубок чемпионов,1
4,премьер-лига,1


In [10]:
teams = []
teams1 = []
teams2 = []

for team1 in df['team_of_season_1_name_in_russian']:
    teams1.append(team1)
for team2 in df['team_of_season_2_name_in_russian']:
    teams2.append(team2)
teams = teams1 + teams2 

teams_dict = {}
for team in teams:
    team = team.lower()
    teams_dict[team] = 'команда'

In [11]:
newdfteam = pd.DataFrame(list(teams_dict.items()), columns=['Word', 'Value'])

In [12]:
newdfteam['Value'] = 1

In [13]:
newdfteam.head(10)

,Word,Value
0,турция,1
1,колумбия,1
2,аз алкмаар,1
3,флора,1
4,валенсия,1
5,реал мадрид,1
6,ахмат,1
7,арсенал,1
8,барселона,1
9,беларусь,1


In [14]:
newdfteam.describe()

,Value
count,4449.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [15]:
words_dict = {}
for text in df['result.tsv'][:15]:
    texts = normalize(text)
    for word in texts:
        words_dict[word] = 'слово'


In [16]:
newdfword = pd.DataFrame(list(words_dict.items()), columns=['Word', 'Value'])

In [17]:
newdfword.head()

,Word,Value
0,сборная,слово
1,турции,слово
2,продолжает,слово
3,свою,слово
4,дорогу,слово


In [18]:
newdfword['Value'] = 0

In [19]:
newdf = newdfteam.append(newdfword)

In [20]:
newdf

,Word,Value
0,турция,1
1,колумбия,1
2,аз алкмаар,1
3,флора,1
4,валенсия,1
5,реал мадрид,1
6,ахмат,1
7,арсенал,1
8,барселона,1
9,беларусь,1


In [21]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [63]:
training_data, testing_data = train_test_split(newdf, random_state = 2000)

In [64]:
y_train = training_data['Value'].values
y_test = testing_data['Value'].values

In [65]:
len(y_test)

1576

In [66]:
words_train = training_data['Word'].values
words_test = testing_data['Word'].values

In [67]:
len(words_test)

1576

In [68]:
vectorizer = CountVectorizer()

In [69]:
X_train = vectorizer.fit_transform(words_train)
X_test = vectorizer.transform(words_test)

In [70]:
clf = LogisticRegression()

In [71]:
clf.fit(X, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [72]:
preds = clf.predict(X_test)

In [73]:
print(classification_report(y_test, preds))

             precision    recall  f1-score   support

          0       0.82      0.02      0.04       485
          1       0.70      1.00      0.82      1091

avg / total       0.73      0.70      0.58      1576

